# RI SPARQL Dashboard
## Contents
### Introduction
With creating a Triplestore in the RDF Serialization Format Turtle from data from the RI indexes, Regests and Wikidata, the resulting dataset can be queried with simple and complex queries in the SPARQL Query Language.
#### Analysis
This Dashboard is intended to show some simple queries and visualizations that can be done based on the RI data in an LOD format such as Turtle, accessed with the SPARQL Query Language. Since the data isn't fully developed or even accounted for, the analysis at this stage is more of a showcase of what could be.
#### Exploration
Way more fruitful for now is an exploratory approach to the dataset, i.e. use the data already developed to develop the rest of the data. For example, by connecting the Turtle Data to a Finite State Machine for Named Entity Recognition, a much quicker and more reliable identification of entities will be possible.
### Usage
To run a cell, select it and press Run or hit Ctrl+Enter. For any of the Queries to run, Libraries and the Graph have to be loaded (the second and third cell.)
### Libraries and Functions
### Graph
### Queries
* #### [Count Properties](#countProperties)
* #### [Recurring Entities](#recurringEntities)
* #### [Occupations and Positions Held by Families](#occupationsFamilies)
* #### [Family Places Map](#familyPlacesMap)
* #### [Issuance Places Map](#issuancePlacesMap)
* #### [Occupations Places Map](#occupationsPlacesMap)


## Libraries and Functions

In [ ]:
#Install Additional Libraries
import sys
#!{sys.executable} -m pip install rdflib
#!{sys.executable} -m pip install folium
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install numpy

In [ ]:
#Libraries
import rdflib
import folium
from importlib import reload
#SPARQL Functions
import sparqlFunctions as sparql

In [ ]:
#Reload SPARQL Functions
reload(sparql)

## Graph

In [ ]:
#Load Graph
#Graph Path
graphPath = "rdf.turtle"
#Init Graph
g = rdflib.Graph()
#Parse Graph Data
g.parse(graphPath, format="turtle")

## Queries
<a id="countProperties"></a>
### Count Properties

In [ ]:
#Count Properties
##Instance of
instanceOf = [row for row in sparql.count(g, "P31") if row[1] > 10]
##Occupations
occupations = [row for row in sparql.count(g, "P106") if row[1] > 10]
##Families
families = [row for row in sparql.count(g, "P53") if row[1] > 10]

print("Instance Of")
for row in instanceOf:
    print(" |".join([str(element) for element in row]))
    
print("Occupations")
for row in occupations:
    print(" |".join([str(element) for element in row]))
    
print("Families")
for row in families:
    print(" |".join([str(element) for element in row]))

<a id="recurringEntities"></a>
### Recurring Entities

In [ ]:
#Recurring Entities
recurringEntities = [[row[0].split("/")[-1]] for row in sparql.recurringEntities(g) if row[1] >= 3]

for row in recurringEntities:
    row.extend([entry[0].split("/")[-1] for entry in sparql.associatedEntries(g, row[0])])
    
with open("recurringEntities.csv", "w", encoding="utf-8") as file:
    for row in recurringEntities:
        file.write("|".join(row) + "\n")

<a id="occupationsFamilies"></a>
### Occupations and Positions Held by Families

In [ ]:
#Occupations by Family
#Habsburg
occupationsHabsburg = [
                        row for row 
                        in sparql.occupationByFamily(g, "Q65968")
                        if row[1] >= 5
                      ]
#Wittelsbach
occupationsWittelsbach = [
                        row for row 
                        in sparql.occupationByFamily(g, "Q131621")
                        if row[1] >= 5
                      ]
#Hohenzollern
occupationsHohenzollern = [
                        row for row 
                        in sparql.occupationByFamily(g, "Q83969")
                        if row[1] >= 5
                      ]

print("Habsburg")
for row in occupationsHabsburg:
    print(" |".join([str(element) for element in row]))
    
print("Wittelsbach")
for row in occupationsWittelsbach:
    print(" |".join([str(element) for element in row]))

print("Hohenzollern")
for row in occupationsHohenzollern:
    print(" |".join([str(element) for element in row]))

<a id="familyPlacesMap"></a>
### Family Places Map
Map of places where family members of the respective family are mentioned or have received charters

In [ ]:
#Family Places Map
m = folium.Map(location=[49.455556, 11.078611], zoom_start=5)

Habsburg = sparql.familyPlaces(g, "Q65968")
Wittelsbach = sparql.familyPlaces(g, "Q131621")
Hohenzollern = sparql.familyPlaces(g, "Q83969")



#Habsburg
for row in Habsburg:
    folium.CircleMarker(
    radius=row[3]/10,
    location=[row[2], row[1]],
    popup=row[0],
    color='crimson',
    fill=True,
    ).add_to(m)

#Wittelsbach
for row in Wittelsbach:
    folium.CircleMarker(
    radius=row[3]/10,
    location=[row[2], row[1]],
    popup=row[0],
    color='#0099cc',
    fill=True,
    ).add_to(m)

#Hohenzollern
for row in Hohenzollern:
    folium.CircleMarker(
    radius=row[3]/10,
    location=[row[2], row[1]],
    popup=row[0],
    color='green',
    fill=True,
    ).add_to(m)
    
m

<a id="issuancePlacesMap"></a>
### Issuance Places Map
Map of issuance places by department

In [ ]:
#Issuance Places Map
m = folium.Map(location=[49.455556, 11.078611], zoom_start=5)

F3 = sparql.issuancePlaces(g, "Q97879676")
LDB = sparql.issuancePlaces(g, "Q97883245")
H7 = sparql.issuancePlaces(g, "Q98414644")

F3_SUM = sum([row[3] for row in F3])
LDB_SUM = sum([row[3] for row in LDB])
H7_SUM = sum([row[3] for row in H7])

#F3
for row in F3:
    folium.CircleMarker(
    radius=(row[3]/F3_SUM)*100,
    location=[row[2], row[1]],
    popup=row[0],
    color='crimson',
    fill=True,
    ).add_to(m)

#LDB
for row in LDB:
    folium.CircleMarker(
    radius=(row[3]/LDB_SUM)*100,
    location=[row[2], row[1]],
    popup=row[0],
    color='#0099cc',
    fill=True,
    ).add_to(m)

#H7
for row in H7:
    folium.CircleMarker(
    radius=(row[3]/H7_SUM)*100,
    location=[row[2], row[1]],
    popup=row[0],
    color='green',
    fill=True,
    ).add_to(m)
    
m

<a id="occupationsPlacesMap"></a>
### Occupations Places Map
Map of places where people with the specified occupation received charters

In [ ]:
#aristocrats = sparql.occupationByIssuancePlaces(g, "Q2478141")
#priests = sparql.occupationByIssuancePlaces(g, "Q250867")
#politicians = sparql.occupationByIssuancePlaces(g, "Q82955")

In [ ]:
#Occupations Places Map
#Loading Premade Data in because Query takes a long time
with open("aristocratsIssuancePlaces.csv", "r", encoding="utf-8") as fileAristocrats, \
     open("priestsIssuancePlaces.csv", "r", encoding="utf-8") as filePriests, \
     open("politiciansIssuancePlaces.csv", "r", encoding="utf-8") as filePoliticians:
    
    aristocrats = [line.strip("\n").split("|") for line in fileAristocrats]
    priests = [line.strip("\n").split("|") for line in filePriests]
    politicians = [line.strip("\n").split("|") for line in filePoliticians]

#Summarize Count
AR_SUM = sum([int(row[3]) for row in aristocrats[1:]])
PR_SUM = sum([int(row[3]) for row in priests[1:]])
PO_SUM = sum([int(row[3]) for row in politicians[1:]])

#Map
m = folium.Map(location=[49.455556, 11.078611], zoom_start=5)

#Aristocrats
for row in aristocrats[1:]:
    folium.CircleMarker(
    radius=(int(row[3])/AR_SUM)*100,
    location=[float(row[2]), float(row[1])],
    popup=row[0],
    color='crimson',
    fill=True,
    ).add_to(m)

#Priests
for row in priests[1:]:
    folium.CircleMarker(
    radius=(int(row[3])/PR_SUM)*100,
    location=[float(row[2]), float(row[1])],
    popup=row[0],
    color='#0099cc',
    fill=True,
    ).add_to(m)

#Politicians
for row in politicians[1:]:
    folium.CircleMarker(
    radius=(int(row[3])/PO_SUM)*100,
    location=[float(row[2]), float(row[1])],
    popup=row[0],
    color='green',
    fill=True,
    ).add_to(m)
    
m